In [17]:
#!/usr/bin/python3

import os, sys
import cv2
import numpy as np
import glob

from tensorflow.lite.python.interpreter import Interpreter

MODEL_NAME = "../models"
GRAPH_NAME = "pose_detection.tflite"
# LABELMAP_NAME = "labelmap.txt"
min_conf_threshold = float(0.5)

IM_NAME = "images/zidane.jpg"
IM_DIR = None

CWD_PATH = os.getcwd()
if IM_DIR:
    PATH_TO_IMAGES = os.path.join(CWD_PATH,IM_DIR)
    images = glob.glob(PATH_TO_IMAGES + '/*')
elif IM_NAME:
    PATH_TO_IMAGES = os.path.join(CWD_PATH,IM_NAME)
    images = glob.glob(PATH_TO_IMAGES)
print("images: ", images)

PATH_TO_CKPT = os.path.join(CWD_PATH,MODEL_NAME,GRAPH_NAME)
# PATH_TO_LABELS = os.path.join(CWD_PATH,MODEL_NAME,LABELMAP_NAME)

# with open(PATH_TO_LABELS, 'r') as f:
#     labels = [line.strip() for line in f.readlines()]

# if labels[0] == '???':
#     del(labels[0])

interpreter = Interpreter(model_path=PATH_TO_CKPT)
interpreter.allocate_tensors()

# Get model details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
height = input_details[0]['shape'][1]
width = input_details[0]['shape'][2]
print("output result number: ",output_details.__len__())
print("input_details: ", input_details)
print("output_details: ", output_details)

floating_model = (input_details[0]['dtype'] == np.float32)

input_mean = 127.5
input_std = 127.5

image_path = images[0]

# Load image and resize to expected shape [1xHxWx3]
image = cv2.imread(image_path)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
imH, imW, _ = image.shape 
image_resized = cv2.resize(image_rgb, (width, height))
input_data = np.expand_dims(image_resized, axis=0)

# Normalize pixel values if using a floating model (i.e. if model is non-quantized)
if floating_model:
    input_data = (np.float32(input_data) - input_mean) / input_std

# Perform the actual detection by running the model with the image as input
# interpreter.set_tensor(input_details[0]['index'],input_data)
interpreter.invoke()

# Retrieve detection results
boxes = interpreter.get_tensor(output_details[0]['index']) # Bounding box coordinates of detected objects
classes = interpreter.get_tensor(output_details[1]['index']) # Class index of detected objects


print("boxes len is :", len(boxes[0][0]))
print("classes len is: ",len(classes[0][0]))




print("boxes =====================")
for box in boxes[0]:
    print(box)
    print("\n")
print("classes =====================")
for classe in classes[0]:
    print(classe)
    print("\n")




images:  ['/home/thundersoft/Desktop/Q4/Demo/run_model/images/zidane.jpg']
output result number:  2
input_details:  [{'name': 'input_1', 'index': 0, 'shape': array([  1, 224, 224,   3], dtype=int32), 'shape_signature': array([  1, 224, 224,   3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
output_details:  [{'name': 'Identity', 'index': 441, 'shape': array([   1, 2254,   12], dtype=int32), 'shape_signature': array([   1, 2254,   12], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}, {'name': 'Identity_1', 'index': 429, 'shape': array([   1, 2254,    1], dtype=int32), 'shape_signature': array([   1, 2254,    1], dtype=int32